# Optimize w knapsack 0/1 algorithm

In [17]:
INPUT_FILE = 'data/dataset2.csv'

In [18]:
import pandas as pd

df = pd.read_csv(INPUT_FILE)

# Transform percent to float
df['profit'] = df['profit'].apply(lambda x: x / 100)
df['gain'] = df['price'] * df['profit']

df.index +=1

df.head()

,name,price,profit,gain
1,Share-MOEX,40.60,0.1669,6.776140
2,Share-GBGY,27.08,0.3409,9.231572
3,Share-AXWK,-9.27,0.2719,-2.520513
4,Share-FJTI,33.50,0.2081,6.971350
5,Share-LGDP,15.26,0.0340,0.518840


In [19]:
df.describe()

,price,profit,gain
count,1000.000000,1000.000000,1000.000000
mean,12.613130,0.196603,2.485969
std,16.238445,0.119171,4.113573
min,-9.950000,0.001500,-3.518640
25%,0.000000,0.089750,0.000000
50%,9.370000,0.198100,0.457746
75%,27.162500,0.305725,4.424702
max,51.460000,0.399800,19.441917


### Missing values

In [20]:
df.isna().sum()

name      0
price     0
profit    0
gain      0
dtype: int64

### Duplicated rows

In [21]:
df[df.duplicated()].shape[0]

0

### Incorrect values
Neither price or profit can't be <= 0.

In [22]:
df_OK = df.drop(df[df['price']<=0].index)
df_OK = df_OK.drop(df_OK[df_OK['profit']<=0].index)

In [23]:
df_OK.describe()

,price,profit,gain
count,541.000000,541.000000,541.000000
mean,25.606543,0.195685,5.055476
std,10.498809,0.122011,4.039245
min,0.120000,0.001500,0.010968
25%,18.310000,0.085800,1.703547
50%,26.120000,0.197100,3.996740
75%,33.010000,0.310000,7.796250
max,51.460000,0.399700,19.441917


### round costs to superior int
So that we can use 0/1 knapsack algorithm, and real cost should never be > max allowed.<br>
If action == 0: do not round. Actions == 0 should be deleted first.

In [24]:
# Arrondir à l'entier supérieur sauf si l'action = 0
import math
df_OK.insert(2, "ceil_price", 0)

if df_OK.price.dtype != 'int64':
    df_OK.ceil_price = df_OK.price.apply(lambda x: math.ceil(x))
else:
    df_OK.ceil_price = df_OK.price

In [25]:
def knapsack(w, wt, val):
    """
    Create a dynamic programming table to get the best profit for each cost
    :param w: int, total maximum price.
    :param wt: list of prices.
    :param val: list of gains
    :return: optimal value, dp table (list of lists)
    """
    n = len(val)  # Nb items
    dp = [[0] * (w + 1) for _ in range(n + 1)]  # Fill array with 0

    for i in range(1, n + 1):
        for w_ in range(1, w + 1):
            if wt[i - 1] <= w_:
                dp[i][w_] = max(val[i - 1] + dp[i - 1][w_ - wt[i - 1]], dp[i - 1][w_])
            else:
                dp[i][w_] = dp[i - 1][w_]
    return dp[n][w_], dp

In [26]:
def knapsack_possible_subset(w: int, wt: list, val: list):
    """
    returns one of the optimal subsets.
    :param w: int, total maximum price.
    :param wt: list, the vector of prices. wt[i] is the weight of the i-th item
    :param val: list, the vector of gains. val[i] is the gain of the i-th item
    :return: optimal_val: float, the optimal gain
    opt_set: set, the indices of the optimal subsets
    total_cost: total cost of optimal subset
    """
    optimal_val, dp_table = knapsack(w, wt, val)
    opt_set: set = set()
    nb_items = len(val)
    _make_subset_from_table(dp_table, wt, nb_items, w, opt_set)
    total_cost = sum([wt[i-1] for i in opt_set])
    return optimal_val, total_cost, opt_set

In [27]:
def _make_subset_from_table(dp: list, wt: list, i: int, j: int, optimal_set: set):
    """
    Recursively look for an optimal subset from the dp table
    and the list of prices

    :param dp: list of list (dp table)
    :param wt: list, prices of the items
    :param i: int, index of the current item
    :param j: int, the current possible maximum price
    :param optimal_set: set, optimal subset recursively modified
    by the function.
    :return: None
    """
    if i > 0 and j > 0:
        if dp[i - 1][j] == dp[i][j]:
            _make_subset_from_table(dp, wt, i - 1, j, optimal_set)
        else:
            optimal_set.add(i)
            _make_subset_from_table(dp, wt, i - 1, j - wt[i - 1], optimal_set)

In [28]:
# reindex
df_OK = df_OK.reset_index(drop=True)
df_OK.index +=1

In [29]:
ceil_prices = df_OK.ceil_price.tolist()
gains = df_OK.gain.tolist()

optimal_val, total_cost, example_set = knapsack_possible_subset(500, ceil_prices, gains)

actions = list(example_set)

opt_df = df_OK.loc[actions,:]

In [30]:
total_cost_real = opt_df.price.sum()
best_set = opt_df.name.tolist()

print(f"Meilleure combinaison : {best_set} \n"
      f"Cout: {round(total_cost_real, 2)} €\n"
      f"Gain: {round(optimal_val, 2)} €")

Meilleure combinaison : ['Share-LFXB', 'Share-GEBJ', 'Share-OPBR', 'Share-NDKR', 'Share-PLLK', 'Share-ZOFA', 'Share-PATS', 'Share-FWBE', 'Share-IJFT', 'Share-ANFX', 'Share-JWGF', 'Share-JGTW', 'Share-ZKSN', 'Share-DWSK', 'Share-ECAQ', 'Share-FAPS', 'Share-ALIY'] 
Cout: 493.1 €
Gain: 194.9 €


In [33]:
opt_df

,name,price,ceil_price,profit,gain
130,Share-LFXB,14.83,15,0.3979,5.900857
5,Share-GEBJ,5.87,6,0.3795,2.227665
454,Share-OPBR,39.00,39,0.3895,15.190500
326,Share-NDKR,33.06,34,0.3991,13.194246
524,Share-PLLK,19.94,20,0.3991,7.958054
526,Share-ZOFA,25.32,26,0.3978,10.072296
399,Share-PATS,27.70,28,0.3997,11.071690
528,Share-FWBE,18.31,19,0.3982,7.291042
180,Share-IJFT,40.91,41,0.3889,15.909899
437,Share-ANFX,38.55,39,0.3972,15.312060
